# Summation & LSTM Model - SICK Dataset

The following notebook contains the implementation of the baseline summation model, and the standard LSTM model for the SICK dataset

In [22]:
# Import the necessary libraries

import re
from numpy import array
from numpy import asarray
from numpy import zeros

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.regularizers import L2
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Input, LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /home/aakashj2412/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
# Helper function that cleans the input data and enumerates the labels

def extract(s):
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()

labels = {'ENTAILMENT': 0, 'CONTRADICTION': 1, 'NEUTRAL': 2}

In [24]:
# Function that reads data and parses data from file

datasetPath = '../../Datasets/SICK/SICK.txt'

with open(datasetPath, 'r') as f:
    Rows = [row.split('\t') for row in f.readlines()[1:]]

trainRows = [row for row in Rows if row[11]=='TRAIN\n']
testRows = [row for row in Rows if row[11]=='TEST\n']
validationRows = [row for row in Rows if row[11]=='TRIAL\n']

trainPremises = [extract(row[1]) for row in trainRows if row[3] in labels]
trainHypotheses = [extract(row[2]) for row in trainRows if row[3] in labels]
trainLabels = [labels[row[3]] for row in trainRows if row[3] in labels]

trainData = [trainPremises, trainHypotheses, trainLabels]

testPremises = [extract(row[1]) for row in testRows if row[3] in labels]
testHypotheses = [extract(row[2]) for row in testRows if row[3] in labels]
testLabels = [labels[row[3]] for row in testRows if row[3] in labels]

testData = [testPremises, testHypotheses, testLabels]

validationPremises = [extract(row[1]) for row in validationRows if row[3] in labels]
validationHypotheses = [extract(row[2]) for row in validationRows if row[3] in labels]
validationLabels = [labels[row[3]] for row in validationRows if row[3] in labels]

validationData = [validationPremises, validationHypotheses, validationLabels]

f.close()

In [25]:
# Hyperparameters

maxLen = 32
epochs = 1000
batchSize = 128
gloveDimension = 300
hiddenDimension = 100
regularization = 4e-6

In [26]:
# Tokenizer to generate the vocabulary of the system

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainData[0] + trainData[1])
vocabSize = len(tokenizer.word_index)+1

In [27]:
# Convert the train data to sequences as per the vocabulary
trainData[0] = tokenizer.texts_to_sequences(trainData[0])
trainData[1] = tokenizer.texts_to_sequences(trainData[1])

# Pad or trim all generated sequences to the same max sentence length
trainData[0] = pad_sequences(trainData[0], maxLen, padding='post')
trainData[1] = pad_sequences(trainData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
trainData[2] = tf.keras.utils.to_categorical(trainData[2], num_classes=3)

In [28]:
# Convert the test data to sequences as per the vocabulary
testData[0] = tokenizer.texts_to_sequences(testData[0])
testData[1] = tokenizer.texts_to_sequences(testData[1])

# Pad or trim all generated sequences to the same max sentence length
testData[0] = pad_sequences(testData[0], maxLen, padding='post')
testData[1] = pad_sequences(testData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
testData[2] = tf.keras.utils.to_categorical(testData[2], num_classes=3)

In [29]:
# Convert the validation data to sequences as per the vocabulary
validationData[0] = tokenizer.texts_to_sequences(validationData[0])
validationData[1] = tokenizer.texts_to_sequences(validationData[1])

# Pad or trim all generated sequences to the same max sentence length
validationData[0] = pad_sequences(validationData[0], maxLen, padding='post')
validationData[1] = pad_sequences(validationData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
validationData[2] = tf.keras.utils.to_categorical(validationData[2], num_classes=3)

In [30]:
# Import the GloVe embeddings and generate embeddings dictionary

embeddingsDict = dict()
glovePath = '../../Datasets/GloVe/'
glove = open(f'{glovePath}glove.840B.300d.txt', encoding='utf8')

for line in glove:
    records = line.split()
    word = ''.join(records[:-300])
    vectorDimensions = asarray(records[-300:], dtype='float32')
    embeddingsDict[word] = vectorDimensions

glove.close()

In [31]:
# Iterate through the embeddings and store only those that are present in our vocabulary

embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    vec = embeddingsDict.get(word)
    if vec is not None:
        embeddingsMat[index] = vec

In [32]:
# Define the embedding layer for our baseline RNN model
embed = Embedding(vocabSize, gloveDimension, weights=[embeddingsMat], input_length=maxLen, trainable=False)

# As Premise and Hypothesis are distinct and are to be inputted separately, define two inputs and embed
premise = Input(shape=(maxLen,), dtype='int32')
hypothesis = Input(shape=(maxLen,), dtype='int32')

premInput = embed(premise)
hypoInput = embed(hypothesis)

convert = Dense(hiddenDimension, activation='tanh', input_shape=(gloveDimension,))

premInput = convert(premInput)
hypoInput = convert(hypoInput)

In [33]:
# Once the sentence embeddings have been generated, generate a matrix of dimensions maxLen X gloveDimension
# On adding maxLen, we get a single embedding vector of length gloveDimension

# Alternate between #1 and #2 rnn definitions to switch between summation and LSTM model

# rnn = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=hiddenDimension)
rnn = LSTM(hiddenDimension, dropout=0.2, recurrent_dropout=0.2)

In [34]:
# Apply batch normalization to the two input embeddings separately

premInput = rnn(premInput)
hypoInput = rnn(hypoInput)
premInput = tf.keras.layers.BatchNormalization()(premInput)
hypoInput = tf.keras.layers.BatchNormalization()(hypoInput)

In [35]:
# Joint is a concatenated embeddings layer, generated from the premise and hypothesis inputs
# Dilution of probability 0.2, to assist in regularization
joint = keras.layers.concatenate([premInput, hypoInput])
joint = Dropout(0.2)(joint)
for i in range(3):
    joint = Dense(2*hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(joint)
    joint = Dropout(0.2)(joint)
    joint = tf.keras.layers.BatchNormalization()(joint)

# 3 layers of the TanH activation function, along with L2 regularization.
# The final decision is based on the Softmax function
pred = Dense(3, activation='softmax')(joint)

In [36]:
# Defining the final models input and output format, as well as compilation parameters

model = Model(inputs=[premise, hypothesis], outputs=pred)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

## Summation Model

The following subsection trains the model and reports the findings for the summation model

In [18]:
# Fitting the model using the train data

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=512, epochs=epochs, callbacks=[callback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])

Epoch 1/1000
9/9 [==============================] - 1s 64ms/step - loss: 0.3143 - accuracy: 0.8718 - val_loss: 1.3876 - val_accuracy: 0.6202
Epoch 2/1000
9/9 [==============================] - 1s 69ms/step - loss: 0.3230 - accuracy: 0.8696 - val_loss: 1.1296 - val_accuracy: 0.6404
Epoch 3/1000
9/9 [==============================] - 1s 67ms/step - loss: 0.2991 - accuracy: 0.8815 - val_loss: 1.3448 - val_accuracy: 0.6303
Epoch 4/1000
9/9 [==============================] - 1s 70ms/step - loss: 0.3034 - accuracy: 0.8779 - val_loss: 1.3157 - val_accuracy: 0.6384
Epoch 5/1000
9/9 [==============================] - 1s 66ms/step - loss: 0.3003 - accuracy: 0.8768 - val_loss: 1.2854 - val_accuracy: 0.6323
Epoch 6/1000
9/9 [==============================] - 1s 72ms/step - loss: 0.3095 - accuracy: 0.8698 - val_loss: 1.0823 - val_accuracy: 0.6727
Epoch 7/1000
9/9 [==============================] - 1s 69ms/step - loss: 0.2814 - accuracy: 0.8835 - val_loss: 1.4074 - val_accuracy: 0.6444
Epoch 8/1000


In [19]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

20/20 [==============================] - 0s 12ms/step - loss: 1.2331 - accuracy: 0.7167
Loss =  1.2330666780471802
Acc =  0.7166734337806702


In [20]:
model.save('../Models/summation_sick.h5')

In [21]:
ccc = keras.models.load_model('../Models/summation_sick.h5')
l, a = ccc.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print(a)

20/20 [==============================] - 1s 14ms/step - loss: 1.2331 - accuracy: 0.7167
0.7166734337806702


We can thus observe the following results for the summation model, for the SICK dataset:

- Training Accuracy: 96.31%
- Validation Accuracy: 74.75%
- Test Accuracy: 71.66%

## LSTM Model

The following subsection trains the model and reports the findings for the LSTM model

In [46]:
# # Fitting the model using the train data

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=100)
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=512, epochs=epochs, callbacks=[callback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])


Epoch 1/1000
9/9 [==============================] - 4s 434ms/step - loss: 0.4027 - accuracy: 0.8259 - val_loss: 0.8393 - val_accuracy: 0.7071
Epoch 2/1000
9/9 [==============================] - 4s 411ms/step - loss: 0.4067 - accuracy: 0.8268 - val_loss: 0.9052 - val_accuracy: 0.6687
Epoch 3/1000
9/9 [==============================] - 4s 407ms/step - loss: 0.3772 - accuracy: 0.8403 - val_loss: 0.8527 - val_accuracy: 0.6727
Epoch 4/1000
9/9 [==============================] - 4s 406ms/step - loss: 0.3953 - accuracy: 0.8328 - val_loss: 0.8451 - val_accuracy: 0.6869
Epoch 5/1000
9/9 [==============================] - 4s 415ms/step - loss: 0.3543 - accuracy: 0.8581 - val_loss: 0.8489 - val_accuracy: 0.6768
Epoch 6/1000
9/9 [==============================] - 4s 397ms/step - loss: 0.3773 - accuracy: 0.8450 - val_loss: 0.8411 - val_accuracy: 0.6929
Epoch 7/1000
9/9 [==============================] - 4s 404ms/step - loss: 0.3683 - accuracy: 0.8423 - val_loss: 0.8740 - val_accuracy: 0.6889
Epoch

In [45]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)


20/20 [==============================] - 1s 49ms/step - loss: 1.2887 - accuracy: 0.7265
Loss =  1.288766953086853
Acc =  0.7265700177192688



In [ ]:
model.save('../Models/LSTM_sick.h5')

In [ ]:
ccc = keras.models.load_model('../Models/LSTM_sick.h5')
l, a = ccc.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print(a)

20/20 [==============================] - 3s 109ms/step - loss: 1.2887 - accuracy: 0.7265
0.726457417011261


We can thus observe the following results for the simple LSTM model, for the SICK dataset:

- Training Accuracy: 95.66%
- Validation Accuracy: 73.14%
- Test Accuracy: 72.65%